<a href="https://colab.research.google.com/github/MMoronto/pytorch_exercises/blob/master/Peutingar_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Dictionary

In [2]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━

##Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path1='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.xls'
path2='/content/drive/MyDrive/Colab Notebooks/infra_data/Salient_cement_stats.csv'
fob=open(path2,)
headings=next(fob)
for rec in fob:
  Cement3=rec.split(',')
  print(Cement3)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['"SALIENT CEMENT STATISTICS1', ' 2 "', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['(Thousand metric tons unless otherwise specified)', '', '', '', '', '', '', '\n']
['', '', '', '', '', '', '', '\n']
['', '', '', '2014', '2015', '2016', '2017', '2018\n']
['United States:', '', '', '', '', '', '', '\n']
['Production:', '', '', '', '', '', '', '\n']
['Cement3', '', '', '"82', '535"', '"84', '405"', '"84', '695"', '"86', '356"', '"86', '400"\n']
['Clinker', '', '', '"74', '372"', '"76', '043"', '"75', '633"', '"76', '678"', '"77', '112"\n']
['"Shipments from mills and terminals:3', ' 4"', '', '', '', '', '', '', '\n']
['Quantity', '', '', '"88', '900"', '"92', '000"', '"94', '300"', '"96', '900"', '"98', '500"\n']
['Value5', 'thousand dollars', '', '"8', '940', '000"', '"9', '800', '000"', '"10', '500', '000"', '"11', '300', '000"', '"11'

In [438]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

os.chdir("../../..")

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


In [439]:
# Create a series and let pandas create a default integer index:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [440]:
from pandas.io.stata import DatetimeIndex
# Create a DataFrame by passing a NumPy array with a dateline index using `date_range()` & labeled columns:
dates = pd.date_range("20230101", periods=8)

dates

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06'], dtype='datetime64[ns]', freq='D')

df = pd.DataFrame(np.random.randn(8, 6), index=dates, columns=list("ABCDEF"))

df 

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562


In [441]:
# Create a `DataFrame` by passing a dictionary of objects that can be converted into a series-like structure:

df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20230102"),
        "C": pd.Series(1, index=list(range(4)), dtype="float32"),
        "D": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)

df2

,A,B,C,D,E,F
0,1.0,2023-01-02,1.0,3,test,foo
1,1.0,2023-01-02,1.0,3,train,foo
2,1.0,2023-01-02,1.0,3,test,foo
3,1.0,2023-01-02,1.0,3,train,foo


In [442]:
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object

In [443]:
df2.D

0    3
1    3
2    3
3    3
Name: D, dtype: int32

##Viewing data

In [444]:
df.head()

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250


In [445]:
df.tail()

,A,B,C,D,E,F
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562


In [446]:
df.columns

Index(['A', 'B', 'C', 'D', 'E', 'F'], dtype='object')

In [447]:
df.index

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08'],
              dtype='datetime64[ns]', freq='D')

`DataFrame.to_numpy()` gives a NumPy representation of the underlying data. 

In [448]:
df.to_numpy()

array([[ 0.11844333, -0.19954283,  0.22208692, -1.57595872,  3.08793745,
        -1.57731695],
       [-0.60667502,  2.64165678,  0.42973134,  0.0374205 , -1.42727558,
         0.21486831],
       [ 0.2704733 , -0.10063135,  0.54418421, -0.40482536, -0.52398595,
         1.17333923],
       [-0.72411581, -0.73335622,  1.36895243,  1.61110654, -1.43202641,
        -0.8606863 ],
       [ 0.31431287, -1.07853246,  0.16679405,  0.23034314, -0.53405697,
        -1.44525016],
       [ 0.60709266,  0.93556595,  1.62017292, -0.06513039, -0.34422594,
        -1.26707542],
       [ 0.25329985, -0.26775143, -0.22847373, -0.1319616 ,  1.28687846,
        -0.46420161],
       [ 0.14451394, -0.21763613,  0.65903924, -0.97239265,  0.54220832,
        -0.97156244]])

In [449]:
df2.to_numpy()

array([[1.0, Timestamp('2023-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2023-01-02 00:00:00'), 1.0, 3, 'train', 'foo'],
       [1.0, Timestamp('2023-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2023-01-02 00:00:00'), 1.0, 3, 'train', 'foo']],
      dtype=object)

In [450]:
df.describe()

,A,B,C,D,E,F
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,0.047168,0.122472,0.597811,-0.158925,0.081932,-0.649736
std,0.465093,1.171305,0.619268,0.930652,1.521174,0.935643
min,-0.724116,-1.078532,-0.228474,-1.575959,-1.432026,-1.577317
25%,-0.062836,-0.384153,0.208264,-0.546717,-0.757362,-1.311619
50%,0.198907,-0.208589,0.486958,-0.098546,-0.434106,-0.916124
75%,0.281433,0.158418,0.836518,0.085651,0.728376,-0.294434
max,0.607093,2.641657,1.620173,1.611107,3.087937,1.173339


In [451]:
df

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562


In [452]:
df.T

,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08
A,0.118443,-0.606675,0.270473,-0.724116,0.314313,0.607093,0.253300,0.144514
B,-0.199543,2.641657,-0.100631,-0.733356,-1.078532,0.935566,-0.267751,-0.217636
C,0.222087,0.429731,0.544184,1.368952,0.166794,1.620173,-0.228474,0.659039
D,-1.575959,0.037420,-0.404825,1.611107,0.230343,-0.065130,-0.131962,-0.972393
E,3.087937,-1.427276,-0.523986,-1.432026,-0.534057,-0.344226,1.286878,0.542208
F,-1.577317,0.214868,1.173339,-0.860686,-1.445250,-1.267075,-0.464202,-0.971562


In [453]:
df.sort_index(axis=1, ascending=False)

,F,E,D,C,B,A
2023-01-01,-1.577317,3.087937,-1.575959,0.222087,-0.199543,0.118443
2023-01-02,0.214868,-1.427276,0.037420,0.429731,2.641657,-0.606675
2023-01-03,1.173339,-0.523986,-0.404825,0.544184,-0.100631,0.270473
2023-01-04,-0.860686,-1.432026,1.611107,1.368952,-0.733356,-0.724116
2023-01-05,-1.445250,-0.534057,0.230343,0.166794,-1.078532,0.314313
2023-01-06,-1.267075,-0.344226,-0.065130,1.620173,0.935566,0.607093
2023-01-07,-0.464202,1.286878,-0.131962,-0.228474,-0.267751,0.253300
2023-01-08,-0.971562,0.542208,-0.972393,0.659039,-0.217636,0.144514


In [454]:
df.sort_values(by="B")

,A,B,C,D,E,F
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868


In [455]:
df

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562


##Getting

In [456]:
# Select a single column, which yields a Series, equivalent to 'df.A'
df["A"]

2023-01-01    0.118443
2023-01-02   -0.606675
2023-01-03    0.270473
2023-01-04   -0.724116
2023-01-05    0.314313
2023-01-06    0.607093
2023-01-07    0.253300
2023-01-08    0.144514
Freq: D, Name: A, dtype: float64

Selecting via `[]` `(__getitem__)`, which slices the rows:

In [457]:
df[0:3]

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339


In [458]:
df["20230102":"20230104"]

,A,B,C,D,E,F
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686


In [459]:
df

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562


##Selection by label

In [460]:
df.loc[dates[1]]

A   -0.606675
B    2.641657
C    0.429731
D    0.037420
E   -1.427276
F    0.214868
Name: 2023-01-02 00:00:00, dtype: float64

In [461]:
#Selecting on a multi-axis by label:
df.loc[:, ["A", "D"]]

,A,D
2023-01-01,0.118443,-1.575959
2023-01-02,-0.606675,0.037420
2023-01-03,0.270473,-0.404825
2023-01-04,-0.724116,1.611107
2023-01-05,0.314313,0.230343
2023-01-06,0.607093,-0.065130
2023-01-07,0.253300,-0.131962
2023-01-08,0.144514,-0.972393


In [462]:
# Showing label slicing, both endpoints are included:
df.loc["20230104":"20230106", ["A", "D"]]

,A,D
2023-01-04,-0.724116,1.611107
2023-01-05,0.314313,0.230343
2023-01-06,0.607093,-0.065130


In [463]:
# Reduction in the dimensions of the returned object:
df.loc["20230104", ["A", "D"]]

A   -0.724116
D    1.611107
Name: 2023-01-04 00:00:00, dtype: float64

In [464]:
# For getting a scalar value
df.loc[dates[1], "D"]

0.03742049674271349

In [465]:
# For getting fast access to a scalar value
df.at[dates[1], "D"]

0.03742049674271349

##Selection by position

In [466]:
df.iloc[3]

A   -0.724116
B   -0.733356
C    1.368952
D    1.611107
E   -1.432026
F   -0.860686
Name: 2023-01-04 00:00:00, dtype: float64

In [467]:
df.iloc[3:5, 0:2]

,A,B
2023-01-04,-0.724116,-0.733356
2023-01-05,0.314313,-1.078532


In [468]:
df.iloc[[1, 2, 4], [0, 2]]

,A,C
2023-01-02,-0.606675,0.429731
2023-01-03,0.270473,0.544184
2023-01-05,0.314313,0.166794


In [469]:
# For slicing rows explicitly
df.iloc[1:3, :]

,A,B,C,D,E,F
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339


In [470]:
# For slicing columns explicitly
df.iloc[:, 1:3]

,B,C
2023-01-01,-0.199543,0.222087
2023-01-02,2.641657,0.429731
2023-01-03,-0.100631,0.544184
2023-01-04,-0.733356,1.368952
2023-01-05,-1.078532,0.166794
2023-01-06,0.935566,1.620173
2023-01-07,-0.267751,-0.228474
2023-01-08,-0.217636,0.659039


In [471]:
# For getting values explicitly
df.iloc[1, 1]

2.6416567790091707

In [472]:
# For getting fast access to scalar (equivalent to the prior method)
df.iat[1, 1]

2.6416567790091707

## Boolean indexing
Using a single column's values to select data:

In [473]:
df[df["A"] > 0]

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562


In [474]:
# Selecting values from a DataFrame where a boolean condition is met:
df[df > 0]

,A,B,C,D,E,F
2023-01-01,0.118443,NaN,0.222087,NaN,3.087937,NaN
2023-01-02,NaN,2.641657,0.429731,0.037420,NaN,0.214868
2023-01-03,0.270473,NaN,0.544184,NaN,NaN,1.173339
2023-01-04,NaN,NaN,1.368952,1.611107,NaN,NaN
2023-01-05,0.314313,NaN,0.166794,0.230343,NaN,NaN
2023-01-06,0.607093,0.935566,1.620173,NaN,NaN,NaN
2023-01-07,0.253300,NaN,NaN,NaN,1.286878,NaN
2023-01-08,0.144514,NaN,0.659039,NaN,0.542208,NaN


In [475]:
df

,A,B,C,D,E,F
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562


In [476]:
# Using the isin() method for filtering:
df2 = df.copy()

df2["G"] = ["one", "one", "two", "three", "four", "three", "five", "four"]

df2

,A,B,C,D,E,F,G
2023-01-01,0.118443,-0.199543,0.222087,-1.575959,3.087937,-1.577317,one
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868,one
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339,two
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686,three
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250,four
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075,three
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202,five
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562,four


In [477]:
df2["G"] = ["one", "one", "two", "three", "four", "three", "five", "four"]

df2[df2["G"].isin(["two", "four"])]

,A,B,C,D,E,F,G
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339,two
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250,four
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562,four


## Setting

Setting a new column automatically aligns the data by the indexes:

In [478]:
s1 = pd.Series([1, 2, 3, 4, 5, 6], index=pd.date_range("20230102", periods=6))
s1

2023-01-02    1
2023-01-03    2
2023-01-04    3
2023-01-05    4
2023-01-06    5
2023-01-07    6
Freq: D, dtype: int64

In [479]:
df["G"] = s1

In [480]:
# Setting values by label:
df.at[dates[0], "A"] = 0

In [481]:
# Setting values by position:
df.iat[0, 1] = 0

In [482]:
df

,A,B,C,D,E,F,G
2023-01-01,0.000000,0.000000,0.222087,-1.575959,3.087937,-1.577317,NaN
2023-01-02,-0.606675,2.641657,0.429731,0.037420,-1.427276,0.214868,1.0
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,-0.523986,1.173339,2.0
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,-1.432026,-0.860686,3.0
2023-01-05,0.314313,-1.078532,0.166794,0.230343,-0.534057,-1.445250,4.0
2023-01-06,0.607093,0.935566,1.620173,-0.065130,-0.344226,-1.267075,5.0
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,1.286878,-0.464202,6.0
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,0.542208,-0.971562,NaN


In [483]:
# Setting by assigning with a NumPy array:
df.loc[:, "E"] = np.array([5] * len(df))

In [484]:
# The result of the prior setting operations:
df

,A,B,C,D,E,F,G
2023-01-01,0.000000,0.000000,0.222087,-1.575959,5,-1.577317,NaN
2023-01-02,-0.606675,2.641657,0.429731,0.037420,5,0.214868,1.0
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,5,1.173339,2.0
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,5,-0.860686,3.0
2023-01-05,0.314313,-1.078532,0.166794,0.230343,5,-1.445250,4.0
2023-01-06,0.607093,0.935566,1.620173,-0.065130,5,-1.267075,5.0
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,5,-0.464202,6.0
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,5,-0.971562,NaN


In [485]:
# A where operation with setting:

df2 = df.copy()

df2[df2 > 0] = -df2

df2

,A,B,C,D,E,F,G
2023-01-01,0.000000,0.000000,-0.222087,-1.575959,-5,-1.577317,NaN
2023-01-02,-0.606675,-2.641657,-0.429731,-0.037420,-5,-0.214868,-1.0
2023-01-03,-0.270473,-0.100631,-0.544184,-0.404825,-5,-1.173339,-2.0
2023-01-04,-0.724116,-0.733356,-1.368952,-1.611107,-5,-0.860686,-3.0
2023-01-05,-0.314313,-1.078532,-0.166794,-0.230343,-5,-1.445250,-4.0
2023-01-06,-0.607093,-0.935566,-1.620173,-0.065130,-5,-1.267075,-5.0
2023-01-07,-0.253300,-0.267751,-0.228474,-0.131962,-5,-0.464202,-6.0
2023-01-08,-0.144514,-0.217636,-0.659039,-0.972393,-5,-0.971562,NaN


##Missing data
pandas primarily uses the value `np.nan` to represent missing data. It is by default not included in computations.

Reindexing allows you to change/add/delete the index on a specified axis. This returns a copy of the data:

In [486]:
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ["G"])

df1.loc[dates[0] : dates[1], "G"] = 1

df1

,A,B,C,D,E,F,G,G
2023-01-01,0.000000,0.000000,0.222087,-1.575959,5,-1.577317,1.0,1.0
2023-01-02,-0.606675,2.641657,0.429731,0.037420,5,0.214868,1.0,1.0
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,5,1.173339,2.0,2.0
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,5,-0.860686,3.0,3.0


`DataFrame.dropna()` drops any rows that have missing data:

In [487]:
df1.dropna(how="any")

,A,B,C,D,E,F,G,G
2023-01-01,0.000000,0.000000,0.222087,-1.575959,5,-1.577317,1.0,1.0
2023-01-02,-0.606675,2.641657,0.429731,0.037420,5,0.214868,1.0,1.0
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,5,1.173339,2.0,2.0
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,5,-0.860686,3.0,3.0


`DataFrame.fillna()` fills missing data:

In [488]:
df1.fillna(value=0)

,A,B,C,D,E,F,G,G
2023-01-01,0.000000,0.000000,0.222087,-1.575959,5,-1.577317,1.0,1.0
2023-01-02,-0.606675,2.641657,0.429731,0.037420,5,0.214868,1.0,1.0
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,5,1.173339,2.0,2.0
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,5,-0.860686,3.0,3.0


`isna()` gets the boolean mask where values are `nan`:

In [489]:
pd.isna(df1)

,A,B,C,D,E,F,G,G
2023-01-01,False,False,False,False,False,False,False,False
2023-01-02,False,False,False,False,False,False,False,False
2023-01-03,False,False,False,False,False,False,False,False
2023-01-04,False,False,False,False,False,False,False,False


##Operations

In [490]:
df

,A,B,C,D,E,F,G
2023-01-01,0.000000,0.000000,0.222087,-1.575959,5,-1.577317,NaN
2023-01-02,-0.606675,2.641657,0.429731,0.037420,5,0.214868,1.0
2023-01-03,0.270473,-0.100631,0.544184,-0.404825,5,1.173339,2.0
2023-01-04,-0.724116,-0.733356,1.368952,1.611107,5,-0.860686,3.0
2023-01-05,0.314313,-1.078532,0.166794,0.230343,5,-1.445250,4.0
2023-01-06,0.607093,0.935566,1.620173,-0.065130,5,-1.267075,5.0
2023-01-07,0.253300,-0.267751,-0.228474,-0.131962,5,-0.464202,6.0
2023-01-08,0.144514,-0.217636,0.659039,-0.972393,5,-0.971562,NaN


In [491]:
df.mean()

A    0.032363
B    0.147414
C    0.597811
D   -0.158925
E    5.000000
F   -0.649736
G    3.500000
dtype: float64